## Representation of Tensor products
Perform tensor product of two vectors and represent.
Create any operator and execute on state vector and finally do the partial measure
Derive the matrix representation of CSWAP and also the Dirac notation
Derive the metric representation for Toffoli operation and also the Dirac notation

In [1]:
# Re-run the Python code for tensor products, operators, partial measurement, CSWAP, and Toffoli.
import numpy as np
from math import sqrt
from pprint import pprint

# --- Utilities ---
def ket0(): return np.array([1,0], dtype=complex)
def ket1(): return np.array([0,1], dtype=complex)

def kron(*states):
    """Tensor (Kronecker) product of multiple arrays"""
    result = states[0]
    for s in states[1:]:
        result = np.kron(result, s)
    return result

def pretty_vec(v):
    return np.array2string(v.flatten(), precision=4, suppress_small=True)

def pretty_mat(m):
    return np.array2string(m, precision=4, suppress_small=True)

def basis_n_qubits(n):
    """Return computational basis for n qubits as list of column vectors"""
    return [np.eye(2**n, dtype=complex)[:,i].reshape(-1,1) for i in range(2**n)]

def idx_to_bitstr(idx, n):
    return format(idx, '0{}b'.format(n))

# --- 1) Tensor product of two vectors ---
v = np.array([1+0j, 2+0j])   # example vector in C^2
w = np.array([3+0j, 4+0j])   # example vector in C^2
v = v.reshape(-1,1); w = w.reshape(-1,1)
tensor_vw = kron(v, w)  # column vector in C^4

print("1) Tensor product of v and w:")
print("v =", v.flatten())
print("w =", w.flatten())
print("v ⊗ w =", pretty_vec(tensor_vw))
print("\n(Interpretation: v⊗w = [v0*w0, v0*w1, v1*w0, v1*w1]^T)\n")

# --- 2) Create an operator and execute on a state vector, then partial measurement ---
# We'll construct a 2-qubit operator: Controlled-Y (control on qubit 0, target qubit 1) as example operator.
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
I = np.eye(2, dtype=complex)
# Controlled-Y: |0><0| ⊗ I + |1><1| ⊗ Y
proj0 = np.array([[1,0],[0,0]], dtype=complex)
proj1 = np.array([[0,0],[0,1]], dtype=complex)
CY = np.kron(proj0, I) + np.kron(proj1, Y)

# Prepare a 2-qubit state: (|0> + |1>)/sqrt(2) on qubit A tensor (|0> + |1>)/sqrt(2) on qubit B = |++>
plus = (1/np.sqrt(2)) * np.array([1,1], dtype=complex).reshape(-1,1)
state_2q = kron(plus, plus)  # 4x1 vector

# Apply operator
final_state = CY @ state_2q
# Normalize (should be unitary so stays normalized)
norm = np.linalg.norm(final_state)
final_state = final_state / norm

print("2) Two-qubit operator (Controlled-Y) applied to |++>")
print("CY matrix:\n", pretty_mat(CY))
print("Initial |++> state:", pretty_vec(state_2q))
print("Final state after CY:", pretty_vec(final_state))

# Partial measurement on first qubit (qubit A):
amps = final_state.flatten()
prob0 = np.sum(np.abs(amps[[0,1]])**2)
prob1 = np.sum(np.abs(amps[[2,3]])**2)
post0 = np.zeros_like(final_state); post0[[0,1]] = final_state[[0,1]]
post1 = np.zeros_like(final_state); post1[[2,3]] = final_state[[2,3]]
if prob0 > 0: post0 = post0 / np.sqrt(prob0)
if prob1 > 0: post1 = post1 / np.sqrt(prob1)

print("\nPartial measurement on first qubit (qubit A):")
print("Prob(qubit A = 0) =", prob0)
print("Post-measurement (A=0) state (normalized):", pretty_vec(post0))
print("Prob(qubit A = 1) =", prob1)
print("Post-measurement (A=1) state (normalized):", pretty_vec(post1))

# Reduced density matrix of second qubit (after application) via partial trace
rho = final_state @ final_state.conj().T  # 4x4 density matrix
def partial_trace(rho_4):
    rho_B = np.zeros((2,2), dtype=complex)
    for i in range(2):
        for j in range(2):
            s = 0+0j
            for k in range(2):
                s += rho_4[2*k + i, 2*k + j]
            rho_B[i,j] = s
    return rho_B
rho_B = partial_trace(rho)
print("\nReduced density matrix of qubit B (after CY):\n", pretty_mat(rho_B))

# --- 3) CSWAP (Fredkin) matrix and Dirac notation ---
n = 3
dim = 2**n
CSWAP = np.zeros((dim, dim), dtype=complex)
for idx in range(dim):
    bits = idx_to_bitstr(idx, n)
    a, b, c = bits[0], bits[1], bits[2]
    if a == '0':
        CSWAP[idx, idx] = 1
    else:
        swapped_bits = a + c + b
        j = int(swapped_bits, 2)
        CSWAP[j, idx] = 1
print("\n3) CSWAP matrix (8x8):\n", pretty_mat(CSWAP))

cswap_dirac = []
for idx in range(dim):
    row = np.where(np.abs(CSWAP[:, idx]) > 0.5)[0][0]
    cswap_dirac.append(f"|{idx_to_bitstr(row,3)}> <{idx_to_bitstr(idx,3)}|")
print("\nCSWAP in Dirac (sum of basis mappings):")
print(" +\n ".join(cswap_dirac))

# --- 4) Toffoli (CCNOT) matrix and Dirac notation ---
TOFF = np.zeros((dim, dim), dtype=complex)
for idx in range(dim):
    bits = idx_to_bitstr(idx, n)
    a, b, c = bits[0], bits[1], bits[2]
    if a == '1' and b == '1':
        flipped_bits = a + b + ('1' if c=='0' else '0')
        j = int(flipped_bits, 2)
        TOFF[j, idx] = 1
    else:
        TOFF[idx, idx] = 1

print("\n4) Toffoli (CCNOT) matrix (8x8):\n", pretty_mat(TOFF))

toff_dirac = []
for idx in range(dim):
    row = np.where(np.abs(TOFF[:, idx]) > 0.5)[0][0]
    toff_dirac.append(f"|{idx_to_bitstr(row,3)}> <{idx_to_bitstr(idx,3)}|")
print("\nToffoli in Dirac (sum of basis mappings):")
print(" +\n ".join(toff_dirac))

# Verify involution property
I8 = np.eye(8, dtype=complex)
print("\nCSWAP^2 == I ? ->", np.allclose(CSWAP @ CSWAP, I8))
print("Toffoli^2 == I ? ->", np.allclose(TOFF @ TOFF, I8))

# Expose for interactive use
outputs = {
    "v": v, "w": w, "v_tensor_w": tensor_vw,
    "CY": CY, "initial_2q": state_2q, "final_state_2q": final_state,
    "post0": post0, "post1": post1, "rho_B": rho_B,
    "CSWAP": CSWAP, "CSWAP_dirac": cswap_dirac,
    "TOFF": TOFF, "TOFF_dirac": toff_dirac
}

print("\nFinished successfully. Variables are available in `outputs` dict.")



1) Tensor product of v and w:
v = [1.+0.j 2.+0.j]
w = [3.+0.j 4.+0.j]
v ⊗ w = [3.+0.j 4.+0.j 6.+0.j 8.+0.j]

(Interpretation: v⊗w = [v0*w0, v0*w1, v1*w0, v1*w1]^T)

2) Two-qubit operator (Controlled-Y) applied to |++>
CY matrix:
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]]
Initial |++> state: [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
Final state after CY: [0.5+0.j  0.5+0.j  0. -0.5j 0. +0.5j]

Partial measurement on first qubit (qubit A):
Prob(qubit A = 0) = 0.5
Post-measurement (A=0) state (normalized): [0.7071+0.j 0.7071+0.j 0.    +0.j 0.    +0.j]
Prob(qubit A = 1) = 0.5
Post-measurement (A=1) state (normalized): [0.+0.j     0.+0.j     0.-0.7071j 0.+0.7071j]

Reduced density matrix of qubit B (after CY):
 [[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]

3) CSWAP matrix (8x8):
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.